 # UIB 11762 CBIR Assignment 2: Image Indexing
 ---

 **Before you begin, please put your name and DNI here:**

 > Marc BALLE SANCHEZ, 43213875A

### Apuntes importantes 

1. Para la ejecución de esta práctica he tenido que migrar a un *entorno local* ya que la plataforma Google Colab presentaba irregularidades respecto a los tiempos de ejecución de mismo bloques de código. Así pues, todos aquellos datos necesarios para la ejecución de la práctica han sido descargados previamente y se han eliminado aquellas celdas encargadas de ello. 

2. Esta práctica es una segunda versión de la que ya se entregó. En esta ocasión soluciono un problema que se tenía con el TF-IDF. Los detalles se comentan en la sección pertinente.  

3. A parte de imprimir datos de interés para la construcción de conclusiones y respuestas, también se han impreso al final de cada apartado los valores de una serie de listas que, en un principio, se concibieron para generar una serie de plots. En la sección del BoW esta impresión es excesivamente grande. Esta información es irrelevante y no debe ser tenida en cuenta.  

In [1]:
# Setup code for this assignment
import cv2
import scipy.cluster.vq as vq
import numpy as np
import os
import time
import copy
from tqdm.notebook import tqdm
import pickle
from matplotlib import pyplot as plt

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
get_ipython().run_line_magic('matplotlib', 'inline')
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'


 > Please, justify all of your answers, graphically wherever possible.
 >
 > Remember that this notebook will be considered as a report to the work done during the assignment.

 ## Introduction
 ---
 In this assignment, you will implement and evaluate different methods for indexing images. As usual during this course, we will use the [INRIA Holidays](http://lear.inrialpes.fr/people/jegou/data.php) dataset. **Check the Assignment 1 to further information about the dataset.**

 Let's download it:

 A folder named *holidays* is now available in your workspace, containing:
 * A set of 1491 images, and
 * A file called *holidays_images.dat*, which is a list with the names of the 1491 images. This file can be used to generate a ground truth.

 > **NOTE**: If you prefer, you can upload the dataset(s) to your Google Drive and mount it in the notebook as explained [here](https://www.youtube.com/watch?v=IZUz4pRYlus).

 We also need the script to evaluate a CBIR system on this dataset. Remember that the performance is measured computing the **mean average precision** (mAP) over all queries. Let's download this file:

In [2]:
import eval_holidays as ev

 Check the Assignment 1 to remember how to use this script and the different functionalities it offers.

 As we did in the previous assignment, next step is to create four lists:
 - **query_names**: File names of the *query* images.
 - **query_imgs**: *Query* images loaded using OpenCV2.
 - **train_names**: File names of the *train* (database) images.
 - **train_imgs**: *Train* images loaded using OpenCV2.

 The size of the images is reduced in order to speed up the process during the rest of the notebook:

In [3]:
# Separating the dataset into query and train images
query_names = []
query_imgs = []
train_names = []
train_imgs = []

with open('holidays/holidays_images.dat') as f:
    for line in f:
        imname = line.strip()
        imno = int(imname[:-len(".jpg")])
        img = cv2.imread('holidays/' + imname)
        img = cv2.resize(img, None, fx=0.25, fy=0.25, interpolation = cv2.INTER_CUBIC)
        
        # Checking if this is a query image
        if imno % 100 == 0:
            query_names.append(imname)
            query_imgs.append(img)
        else:
            train_names.append(imname)
            train_imgs.append(img)

print(len(query_imgs))
print(len(train_imgs))


500
991


 In this assignment we will create four additional lists:

 - **query_kps**: A list of lists of keypoints (cv2.KeyPoint) extracted from the *query* images.
 - **query_desc**: A list of Numpy arrays including, for each set of keypoints, the SIFT descriptors extracted from the *query* images.
 - **train_kps**: A list of lists of keypoints (cv2.KeyPoint) extracted from the *train* (database) images.
 - **train_desc**: A list of Numpy arrays including, for each set of keypoints, the SIFT descriptors extracted from the *train* images.

 Unlike in Assigment 1, here you will be provided with a set of SIFT descriptors for each image, and, therefore, you do not need to create these lists from scratch. First, let's download the descriptors:

 Now, a new directory called *siftgeo* is in your workspace, containing the set of SIFT descriptors for each image of the dataset. These descriptors are stored in binary format and, thus, you will need some tools to load them. You are also provided with tools for that purpose:

In [4]:
import index_utils as iu

 One loaded, you can call the function `load_SIFT_descriptors` to load the descriptors of a list of images:

In [5]:
query_kps, query_desc = iu.load_SIFT_descriptors(query_names, max_desc=1000)
train_kps, train_desc = iu.load_SIFT_descriptors(train_names, max_desc=1000)

# Some prints
print(len(query_kps))
print(len(train_kps))
print(len(query_desc))
print(len(train_desc))
print(query_desc[0].shape)
print(query_desc[0])


500
991
500
991
(1000, 128)
[[10.  6. 52. ... 15.  4.  0.]
 [16. 50. 12. ... 15.  4.  0.]
 [10. 11. 58. ...  7.  4.  4.]
 ...
 [27. 15.  0. ... 16.  8. 12.]
 [51. 47. 14. ... 35. 26.  0.]
 [ 2. 37. 25. ... 47. 13.  8.]]


 For development purposes, we use the parameter `max_desc` to load a maximum number (1000) of the descriptors. This will speed up the execution of the rest of the notebook, while the decrease in performance will be minimum.

 ## $k$-d Trees
 ---
 In this section you will use a set of randomized $k$-d trees to index the database of images. Given a query image, the best matching image will be the one with the higher number of matches, according to the Nearest Neighbor Distance Ratio (NNDR). Write a function called `build_db_kdtrees` to build a set of randomized $k$-d trees given a set of training descriptors:

 > **Useful links**: [cv2.FlannBasedMatcher](https://docs.opencv.org/3.4/dc/de2/classcv_1_1FlannBasedMatcher.html), [Possible algorithms to create an index](https://docs.opencv.org/4.5.1/db/d18/classcv_1_1flann_1_1GenericIndex.html#a8fff14185f9f3d2f2311b528f65b146c), [Algorithms IDs](https://github.com/opencv/opencv/blob/master/modules/flann/include/opencv2/flann/defines.h#L70)

In [6]:
def build_db_kdtrees(img_names, descs, ntrees = 4):
    '''
    Builds a set of randomized k-d trees.
    
    - img_names: An ordered list with the image names.
    - descs: A list of length len(img_names) where each element is a numpy array 
            of size (ndesc_for_this_image, 128). Each Numpy array i corresponds 
            to the descriptors found on image i.
    - ntrees: Number of trees to train.

    RETURNS: 
    - index: trained FLANN index.
    - id_to_img: An associative list to link every image id to the its real name
            e.g. id_to_img[0] = '100001.jpg', id_to_img[1] = '100002.jpg'.
    '''

    index = []
    id_to_img = []

    ##############################################################################
    # TODO:                                                                      #
    # Write this function according to the description given above.              #
    ##############################################################################
    
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = ntrees)
    search_params = {}  #{} 
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    
    start = time.time()
    flann.add(descs)
    flann.train()
    stop = time.time()

    training_time = stop - start
    
    ##############################################################################
    #                                 END OF YOUR CODE                           #
    ##############################################################################

    #return index#,id_to_img
    return flann, training_time #id_to_img -> es el mismo que train_names



 Next, write a function to search an image in a generic index (database). You should search the descriptors of the query image and obtain their two closest SIFT descriptors in the database. Next, the initial set of matches should be filtered using the NNDR criterion, as in the previous assignment. For each database image, its final score with regard to this query image will be the number of correct matches with this image:

In [7]:

def search_image(descs, index, train_names):
    '''
    Search an image in the index
    
    - descs: A numpy array. This is the set descriptors 
            extracted from the query image.
    - index: FLANN index to search for descriptors.
    - id_to_name: An associative list to link every image index to its real name
            e.g. id_to_img[0] = '100001.jpg', id_to_img[1] = '100002.jpg'

    RETURNS: 
    - An ordered list of similar images, e.g.: ['100101.jpg', '100202.jpg', ...]
    '''

    ##############################################################################
    # TODO:                                                                      #
    # Write this function according to the description given above.              #
    ##############################################################################
    
    ratio = 0.75
    init_dict = [(img_name, 0) for img_name in train_names]
    score = dict(init_dict)
    start = time.time()
    matches = index.knnMatch(descs, k = 2) 
    stop = time.time()
    filt_matches = list(filter(lambda m: m[0].distance < m[1].distance * ratio, matches))
    for match in filt_matches: 
        score[train_names[match[0].imgIdx]]+=1

    #score_cp = copy.deepcopy(score) #elimino imágenes con 0 matches
    #for entry in score_cp.keys(): 
    #    if not score_cp[entry]:
    #        score.pop(entry) 

    values = np.array(list(score.values()))
    imgs_names = list(score.keys())
    index_sort = np.argsort(values)[::-1] #descending order 
    best_imgs = [imgs_names[i] for i in index_sort]
    
    query_time = stop-start
    return best_imgs, query_time

    ##############################################################################
    #                                 END OF YOUR CODE                           #
    ##############################################################################



 Finally, write a function called `compute_mAP`. Given a list of query images and a trained index, this function should return a Python dictionary with the ordered results for each query along with the computed mAP:

In [8]:
def compute_mAP(query_names, query_descs, index, train_names, gt_file = 'holidays/holidays_images.dat'):
    '''
    Perform a search for a list of query images against the database.
    
    - query_names: An ordered list with the names of the query images.
    - query_desc: A list containing numpy arrays of size 
            (ndesc_for_this_image, 128). Each numpy array i corresponds to the 
            descriptors found at image i.
    - index: FLANN index.
    - id_to_name: An associative array to link every image index to its real name
            e.g. id_to_img[0] = '100001.jpg', id_to_img[1] = '100002.jpg'
    - gt_file = Ground truth file. Typically, 'holidays_images.dat'.

    RETURNS: 
    - total_results: A dictionary containing, for each query image, 
            an ordered list of the retrieved images.
    - m_ap: Mean Average Precision averaged over all queries.
    '''
    total_results = {}
    m_ap = 0.0
    query_times = []
    ##############################################################################
    # TODO:                                                                      #
    # Write this function according to the description given above.              #
    ##############################################################################
    
    for query_name, query_desc in zip(query_names, query_descs):
            results, query_time = search_image(query_desc, index, train_names)
            total_results[query_name] = results 
            query_times.append(query_time)
        
    m_ap = ev.compute_mAP(total_results, gt_file)
    
    ##############################################################################
    #                                 END OF YOUR CODE                           #
    ##############################################################################
    
    return total_results, m_ap, np.array(query_times)


 > **Questions**: Search the query images against the database using the `compute_mAP` function.
 >
 > - What is the mAP obtained using this approach?
 
 >   <font color = 'blue'>El mAP obtenido se calcula en base a las imágenes devueltas por la base de datos ordenadas en un orden tal que las primeras sean las más parecidas a la imagen de petición y las últimas las más diferentes. En el caso de los kd-trees, la búsqueda lineal se sustituye por una búsqueda en un árbol binario. En dicho árbol cada nodo hoja es un descriptor de alguna imágen de la base de datos. A cada descriptor de la imagen de petición se le asignan en este caso los dos descriptores más cercanos buscados a lo largo de varios kd-trees. Una vez filtrados dichos emparejamientos según el criterio NNDR, se obtienen las correspondencias entre los descriptores e imágenes de entrenamiento y se actualiza un contador en función de la imagen a la que pertenecen cada descriptor emparejado. Así, las imágenes con más votos son devueltas en las primeras posiciones.</font>
 
 > - Is it stable? Why?
 
 >    <font color = 'blue'>Para comprobar si el mAP devuelto es estable, se ha repetido el proceso de búsqueda 3 veces. Como puede contemplarse en los datos impresos por pantalla, el mAP varía muy poco entre iteraciones. De esta manera, puede concluirse que sí es estable.</font> 
 
 > - Analyze the effect of changing the **number of trees** in terms of mAP and average response time. Some plots here can be useful to justify your answer.
 
 >   <font color = 'blue'>Para contestar a esta cuestión se ha probado a costruir un indice con 3, 4 y 5 árboles. Como puede observase en las impresiones, tanto el mAP como el tiempo total de contrucción y búsqueda de cada índice son muy parecidos. Sí que es cierto que a cuanto más árboles más tarda el proceso completo de contrucción y búsqueda. Al final esto parece lógico ya que tanto la construcción del índice y la indexación de descriptores tarda más a cuántos más árboles tenga uno que referirse o prestar atención (por decirlo de alguna manera). Sin embargo, no parece ser que a cuántos más árboles mejor es el mAP. El mejor valor de dicha métrica se obtiene para 4 árboles, aunque la diferencia con el resto de índices es muy baja, menor al 0,01. Por ello, no puede decirse que sea una diferencia significativa.</font> 
 

In [9]:
################################################################################
# TODO:                                                                        #
# Write here the code required to answer the questions stated above. You can   #
# Add more cells at this point if you need it.                                 #
################################################################################

mean_train_t_kd = 0.0
mean_query_t_kd = 0.0
niters = 3
for i in range(niters):
    index, train_time_kd = build_db_kdtrees(train_names, train_desc, ntrees = 3)
    results, m_ap, query_time_kd = compute_mAP(query_names, query_desc, index, train_names)
    mean_train_t_kd += train_time_kd
    mean_query_t_kd += np.mean(query_time_kd)
    print('Iteration {}\n'.format(i))
    print('MAP: {} \n'.format(m_ap))
    print('Training time: {} secs.'.format(train_time_kd))
    print('Query response time: {} +- {} secs.'.format(np.mean(query_time_kd), np.std(query_time_kd)))
    print('\n\n')
    

print ('Mean training time Kd-trees: {} secs.'.format(mean_train_t_kd/niters))
print('Mean query response time Kd-trees: {} secs'.format(mean_query_t_kd/niters))

num_trees = [3,4,5]
m_ap_list = []
time_tree = []
for tree in num_trees: 
    start = time.time()
    index, _ = build_db_kdtrees(train_names, train_desc, ntrees = tree)
    results, m_ap, _ = compute_mAP(query_names, query_desc, index, train_names)
    stop = time.time()
    m_ap_list.append(m_ap)
    time_tree.append(stop-start)
    print('Number of trees: {}'.format(tree))
    print('MAP: {}\n'.format(m_ap))
    print('Average response time: {} secs.'.format(stop-start))
    print('\n\n')

    
print('Printing variables: \n')
print('m_ap_list: {} \n'.format(m_ap_list))
print('time_tree: {} \n'.format(time_tree))
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

Iteration 0

MAP: 0.6879927943384108 

Training time: 4.547006845474243 secs.
Query response time: 0.06576506376266479 +- 0.01895761496540475 secs.



Iteration 1

MAP: 0.6975660353881761 

Training time: 4.711634159088135 secs.
Query response time: 0.07384067058563232 +- 0.03547091900240361 secs.



Iteration 2

MAP: 0.6839776488979906 

Training time: 4.5130650997161865 secs.
Query response time: 0.06369509649276733 +- 0.01834816880132994 secs.



Mean training time Kd-trees: 4.5905687014261884 secs.
Mean query response time Kd-trees: 0.06776694361368814 secs
Number of trees: 3
MAP: 0.6900373167008104

Average response time: 36.9909930229187 secs.



Number of trees: 4
MAP: 0.6995479688941076

Average response time: 39.15631985664368 secs.



Number of trees: 5
MAP: 0.6922023578233276

Average response time: 42.34748888015747 secs.



Printing variables: 

m_ap_list: [0.6900373167008104, 0.6995479688941076, 0.6922023578233276] 

time_tree: [36.9909930229187, 39.15631985664368, 42.347

 ## Locality Sensitive Hashing (LSH)
 ---
 In this section, you will use LSH to index the database of images. LSH implementation included in OpenCV uses **bit sampling** for **Hamming distance** in order to construct the hash family and, therefore, binary descriptors should be used. Hence, SIFT descriptors are not valid and we need to describe again the images, but using, for instance, ORB.

 In the following cell, write the code required to generate roughly 1500 keypoints / descriptors using ORB for each query / test image:

 > **Useful functions**: [cv2.FlannBasedMatcher](https://docs.opencv.org/3.4/dc/de2/classcv_1_1FlannBasedMatcher.html)

In [10]:
query_kps_orb = []
query_desc_orb = []
train_kps_orb = []
train_desc_orb = []

##############################################################################
# TODO:                                                                      #
# Extract 1500 keypoints / descriptors from every query and test images      # 
# using ORB. Store the results in the list indicated above.                  #
##############################################################################

orb = cv2.ORB_create(nfeatures = 1500, fastThreshold = 50)
for query in query_imgs: 
    kp,des = orb.detectAndCompute(query, mask=None)
    query_kps_orb.append(kp)
    query_desc_orb.append(des)
    
for train in train_imgs: 
    kp,des = orb.detectAndCompute(train, mask=None)
    train_kps_orb.append(kp)
    train_desc_orb.append(des)
    
##############################################################################
#                                 END OF YOUR CODE                           #
##############################################################################

# Show some data
print(len(query_kps_orb[0]))
print(query_desc_orb[0].shape)
print(query_desc_orb[0])


965
(965, 32)
[[182 247  64 ... 214 158 220]
 [ 56 141 104 ...  43 212 234]
 [227 182 190 ...  68 239 223]
 ...
 [ 71 215 226 ... 216 177  29]
 [ 18  40 214 ...   0  41   4]
 [ 42 176  32 ...  15 155 215]]


 Write a function to build a **standard** (*no multi-probe*) LSH index from a set of images:

In [11]:
def build_db_lsh(img_names, descs, tables = 6, hash_size = 12):
    '''
    Index a set of images using LSH.
    
    - img_names: An ordered list with the names of query images.
    - descs: A list containing numpy arrays of size (~1500, 128). Each numpy array
            i corresponds to the descriptors found at image i.
    - tables: Number of hash tables to create.
    - hash_size: Hash length in bits.

    RETURNS: 
    - index: The trained LSH index.
    - id_to_img: An associative list to link every index to the real name of the
        image e.g. id_to_img[0] = '100001.jpg', id_to_img[1] = '100002.jpg'
    '''  
    #index = []
    #id_to_img = []

    ##############################################################################
    # TODO:                                                                      #
    # Write this function according to the description given above.              #
    ##############################################################################
    
    FLANN_INDEX_LSH = 6
    index_params = dict(algorithm = FLANN_INDEX_LSH, table_number = tables, key_size = hash_size, multi_probe_level = 0)
    search_params = {}  #{} 
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    
    start = time.time()
    flann.add(descs)
    flann.train()
    stop = time.time()
    
    training_time = stop - start
    ##############################################################################
    #                                 END OF YOUR CODE                           #
    ##############################################################################

    return flann, training_time


 > **Questions**: Search the query images against the database using the `compute_mAP` function and the LSH index:
 >
 > - What is the mAP obtained using this approach?
 
 > <font color = 'blue'>En este caso, en lugar de buscar en un árbol binario, se busca en tablas de hash donde cada descriptor de entrenamiento es asignado a un determinado código binario empleado una función de hashing. En este caso, dicha función es el _bit sampling_ . Posteriormente, los descriptores de la imagen de petición son transformados a códigos binarios con la misma función. Cada descriptor de petición se empareja con dos descriptores de entrenamiento con el mismo código binario. De nuevo, una vez filtrados dichos emparejamientos, se realiza un conteo de las imágenes de entrenamiento correspondientes a cada descriptor emparejado con los descriptores de petición. Aquellas imágenes con mayor nñumero de descriptores emparejados con la imagen de petición se devuelven las primeras.</font> 
 
 > - Is it stable? Why?
 
 > <font color = 'blue'>De nuevo, se puede decir que el sistema es estable en término de mAP. Se han realizado 3 iteraciones empleando índices contruidos con los mismos parámetros, y los mAP son casi idénticos entre iteraciones.</font>  
 
 > - Analyze the effect of changing the **number of tables** / **hash size** in terms of mAP and average response time. Some plots here can be useful to justify your answer.
 
 > Para contestantar a esta pregunta se han realizado varias pruebas con combinaciones distintas de parámetros. Se hab probado dos números distintos de tablas, 6 y 8, y dos tamaños distintos de código hash binario, 12 y 16. 
 
 > <font color = 'blue'>Como puede observarse en las impresiones, por una parte se concluye que a cuantas más tablas se usen mayor es el tiempo total del proceso (entrenamiento más búsqueda de imágenes). Por otra parte, a mayor tamaño del código binario de hash empleado, mucho menor es dicho tiempo de ejecución, con una reducción entorno al 85%. En cuanto al mAP, en ambos casos es muy semejante, no hay diferencias significativas entre usar más o menos tablas o entre usar un tamaño de código más o menos grande.</font>  
 
 > - Despite the different descriptors used, compare the performance of LSH and randomized $k$-d trees from different points of view (accuracy, memory, training times, querying times, ...). Some plots here can be useful to justify your answer.
 
 > <font color = 'blue'>En esta pregunta me he centrado en analizar los tiempos de entrenamiento de cada índice y los tiempos medios de búsqueda por imagen, así como la desviación que estos presentan respecto de la media para analizar su estabilidad entre búsquedas. Para ello se han determinado estos valores para 3 iteraciones distintas con los mismos parámetros constructivos de los índices, dotando así de mayor signifiancia estadística a las conclusiones extraídas.</font> 
 
 > <font color = 'blue'>Una vez dicho esto, se puede decir que el tiempo de entrenamiento de los kd-trees es mucho mayor al LSH, entorno a 6 veces más rápidos estos útlimos en entrenarse. Sin embargo, en cuanto a indexación de imágenes, se aprecia que los kd-trees son mucho más rápidos que los LSH, con tiempos 6 o 7 veces menores que estos últimos. En cuanto a la estabilidad en los tiempos de búsqueda, se observa que los kd-trees presentan desviaciones estándar proporcionalmente menores a los LSH. Esto últimos presentan desviación de casi el 50% respecto de la media. Por ello, puede decirse que en cuanto a tiempo de indexación, los kd-trees son más estables.</font> 
 
 > <font color = 'blue'>Finalmente también puede realizarse una comparación de mAP. Simplemente comentar que los kd-trees obtienen valores mAP mayores que LSH. Ambos índices son igualmente estables en términos de dicha métrica.</font> 
 
 > <font color = 'blue'>No se han establecido comparaciones a nivel de memoria usada por motivos de destreza técnica. No sabía cómo evaluar dicho aspecto. Aunque basándome en los conceptos teóricos de cada método, me aventuraría a decir que, pese a que ambos métodos necesitan tener todos los descriptores cargados en memoria,  los LSH emplean menos recursos que los kd-trees, ya que estos últimos emplean descriptores en coma flotante,a diferencia de los primeros que emplean descriptores binarios.</font>  

In [13]:
################################################################################
# TODO:                                                                        #
# Write here the code required to answer the questions stated above. You can   #
# Add more cells at this point if you need it.                                 #
################################################################################

mean_query_t_lsh = 0.0
mean_train_t_lsh = 0.0
#Stability
for i in range(niters):
    index_lsh, train_time_lsh = build_db_lsh(train_names, train_desc_orb, tables = 6, hash_size = 12)
    results_lsh, m_ap_lsh, query_time_lsh = compute_mAP(query_names, query_desc_orb, index_lsh, train_names)
    mean_query_t_lsh += np.mean(query_time_lsh)
    mean_train_t_lsh += train_time_lsh
    print('Iteration {}\n'.format(i))
    print('MAP: {} \n'.format(m_ap_lsh))
    print('Training time: {} secs.'.format(train_time_lsh))
    print('Query response time: {} +- {} secs.'.format(np.mean(query_time_lsh), np.std(query_time_lsh)))
    print('\n\n')
    

print ('Mean training time LSH: {} secs.'.format(mean_train_t_lsh/niters))
print('Mean query response time LSH: {} secs'.format(mean_query_t_lsh/niters))


# Response time w/ different paramenters
param_combs = [[6,12], [6,16], [8,12], [8,16]]
m_ap_lsh_list = []
time_lsh = []
for comb in param_combs: 
    start = time.time()
    index_lsh, _ = build_db_lsh(train_names, train_desc_orb, tables = comb[0], hash_size = comb[1])
    results_lsh, m_ap_lsh, _ = compute_mAP(query_names, query_desc_orb, index_lsh, train_names)
    stop = time.time()
    m_ap_lsh_list.append(m_ap_lsh)
    time_lsh.append(stop-start)
    print('Combination -> tables = {}; hash_size = {} \n'.format(comb[0], comb[1]))
    print('MAP: {}\n'.format(m_ap_lsh))
    print('Average response time: {} secs.'.format(stop-start))
    print('\n\n')
    
    
print('Printing variables: \n')
print('m_ap_lsh_list: {} \n'.format(m_ap_lsh_list))
print('time_lsh: {} \n'.format(time_lsh))
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################


Iteration 0

MAP: 0.5570587650087657 

Training time: 0.683635950088501 secs.
Query response time: 0.4261805658340454 +- 0.22208931480614968 secs.



Iteration 1

MAP: 0.5521084270410586 

Training time: 0.7014031410217285 secs.
Query response time: 0.4182029438018799 +- 0.2081014932406212 secs.



Iteration 2

MAP: 0.5515775819022046 

Training time: 0.6714541912078857 secs.
Query response time: 0.49273929500579833 +- 0.24966283784247303 secs.



Mean training time LSH: 0.6854977607727051 secs.
Mean query response time LSH: 0.4457076015472412 secs
Combination -> tables = 6; hash_size = 12 

MAP: 0.5558477789499733

Average response time: 237.3519468307495 secs.



Combination -> tables = 6; hash_size = 16 

MAP: 0.5568846196141212

Average response time: 36.71414113044739 secs.



Combination -> tables = 8; hash_size = 12 

MAP: 0.553578533273073

Average response time: 288.5771508216858 secs.



Combination -> tables = 8; hash_size = 16 

MAP: 0.5493404076574129

Average response tim

 ## Bag of Words (BoW)
 ---

 ### Download Visual Dictionaries
 To use a BoW model, first we need a visual vocabulary. The authors of the INRIA Holidays dataset provide some visual vocabularies, trained using a clustering method (like $k$-means) in a different dataset (Flickr60K).

 First, let's download these vocabulary files:

 A folder named *clust* is now available in your workspace, containing visual vocabularies of 100, 200, 500, 1K, 2K, 5K, 10K, 20K, 50K, 100K and 200K visual words. Again, these are binary files, and therefore we provide you with functions to load and index them:

In [6]:
v = iu.load_visual_vocab("clust/clust_flickr60_k200.fvecs", ntrees=4)

 With this function, the corresponding vocabulary is read and, additionally, a FLANN index structure based on kd-trees is built and returned. This is to allow a fast access when searching for the closest visual words in the vocabulary. More precisely, in this example, 4 trees are constructed using the 200 centroids. Now, given a query descriptor(s), you can use `match` or `knnMatch` methods as usual to search for the closest visual words in the vocabulary.

 ### BoW and Inverted File
 Now, write a class called `BoW` to manage the indexing procedure. This class should make use, in addition to the visual vocabulary, an inverted file to compute similarity scores between images. Apart from the class constructor, write two methods, `build_index` and `search_image`, to build the index using a set of train images and to search a query image in the database, respectively:

In [15]:
class BoW(object):
    '''
    Class to implement the BoW model + Inverted File.
    '''

    def __init__(self, vocab_file):
        '''
        Class constructor. You should load the vocabulary and initialize other stuff
        required for the CBIR system, such as the inverted file structure.
        '''
        self.vocab = iu.load_visual_vocab(vocab_file)
        self.nwords = self.vocab.getTrainDescriptors()[0].shape[0]
        
        ############################################################################
        # TODO:                                                                    #
        # Complete this function as indicated                                      #
        ############################################################################
        init_dict = [(str(i), []) for i in range(self.nwords)] 
        self.inverted = dict(init_dict)
        ############################################################################
        #                                 END OF YOUR CODE                         #
        ############################################################################

    def build_index(self, img_names, img_descs):
        '''
        Build an index from a set of images. Essentially, for each image, you should
            search its descriptors in the index and fill the inverted file structure
            in consequence.

        - img_names: An ordered list with the names of the train images.
        - img_descs: A list containing numpy arrays. Each numpy array i corresponds 
            to the descriptors found at image i.
        '''
        ############################################################################
        # TODO:                                                                    #
        # Write this function according to the description given above.            #
        ############################################################################
        start = time.time()
        for name, descs in zip(img_names, img_descs): 
            matches = self.vocab.match(descs)
            idxs = [match.trainIdx for match in matches]
            idxs_u = np.unique(np.array(idxs)) #para evitar imagenes repetidas en el inverted file
            for idx in list(idxs_u):
                self.inverted[str(idx)].append(name)
        stop = time.time()
        training_time = stop-start
        
        return training_time
    
        ############################################################################
        #                                 END OF YOUR CODE                         #
        ############################################################################

    def search_image(self, descs):
        '''
        Search an image in the index.
    
        - descs: A numpy array. It is the set descriptors extracted 
            from the query image.

        RETURNS:
        - An ordered list of similar images, e.g.: ['100101.jpg', '100202.jpg', ...]
        '''
        ############################################################################
        # TODO:                                                                    #
        # Write this function according to the description given above.            #
        ############################################################################
        
        start = time.time()
        matches = self.vocab.match(descs)
        idxs = [match.trainIdx for match in matches]
        counter = {}
        for idx in idxs: 
            retrieved_imgs = self.inverted[str(idx)]
            for ret_img in retrieved_imgs: 
                if ret_img not in list(counter.keys()): 
                    #counter.setdefault(ret_img, 1) #creo la entrada con valor 1
                    counter[ret_img] = 1
                else: 
                    counter[ret_img] += 1
        
        values = np.array(list(counter.values()))
        imgs_names = list(counter.keys())
        index_sort = np.argsort(values)[::-1] #descending order 
        best_imgs = [imgs_names[i] for i in index_sort]
        stop = time.time()
        query_time = stop-start
        
        return best_imgs, query_time
        ############################################################################
        #                                 END OF YOUR CODE                         #
        ############################################################################


 Finally, as usual, write a function called `compute_mAP` to the compute the performance of the system, given a lists of query images. This function should return a Python dictionary with the ordered results for each query along with the computed mAP:

In [15]:
def compute_mAP(query_names, query_descs, index, gt_file = 'holidays/holidays_images.dat'):
    '''
    Perform a search for a list of query images against the database and evaluates
    the performance of the system.
    
    - query_names: An ordered list with the names of query images.
    - query_descs: A list containing numpy arrays of size 
            (ndesc_for_this_image, 128). Each numpy array i corresponds to the 
            descriptors found at image i.
    - index: Index to search.
    - gt_file = Ground truth file. Typically, 'holidays_images.dat'.

    RETURNS: 
    - total_results: A dictionary containing, for each query image, 
            an ordered list of the retrieved images.
    - m_ap: Mean Average Precision averaged over all queries.
    '''
    total_results = {}
    m_ap = 0.0

    ##############################################################################
    # TODO:                                                                      #
    # Write this function according to the description given above.              #
    ##############################################################################
    query_times = []
    for query_name, query_desc in zip(query_names, query_descs):
            results, query_time = index.search_image(query_desc)
            total_results[query_name] = results 
            query_times.append(query_time)
        
    m_ap = ev.compute_mAP(total_results, gt_file)
    
    ##############################################################################
    #                                 END OF YOUR CODE                           #
    ##############################################################################
    
    return total_results, m_ap, np.array(query_times)


 Using the vocabularies of 200, 2K, 20K and 200K visual words, answer the following questions:
 > **Questions**:
 >
 > - What is the mAP obtained for each visual vocabulary?
 
 > <font color = 'blue'>En este caso, cada descriptor de la imagen de petición se emparejado con una palabra visual. Esta palabra visual no es nada más que un descriptor "abstracto" fruto del proceso de clustering en un espacio de descriptores provenientes de imágenes de entrenamiento. En esta caso concreto, estas imágenes de entrenameinto provienen del dataset Flickr60. Cada palabra visual guarda un registro de aquellas imágenes que contienen dicha palabra visual en una archivo llamado fichero inverso. Así pues, por cada descriptor de petición se registran o se lleva la cuenta de aquellas imágenes enlazadas con la palabra visual emparejada a dicho descriptor. Aquellas imágenes que han aparecido con más frecuencia se retornan las primeras.</font> 
 
 > - Compare the performances obtained on each case. Is a larger vocabulary size always better? Why or why not?
 
 > <font color = 'blue'>En este caso, a mayor número de palabras visuales contiene el diccionario, mejor mAP se obtiene. Además, se observa que los tiempo de petición también disminuyen considerablemente a cuantas más palabras se tengan. Sin embargo, el tiempo de entrenamiento del índice aumenta en este caso. No obstante, esta fase puede hacerse de manera offline, por lo que no condiciona al rendimiento en real-time del modelo. Es por ello que, ateniéndome a este caso particular, sí que parece beneficioso el hecho de usar más palabras en el diccionario en todos los aspectos.  Aun así, en aplicaciones donde los recursos de memoria sean limitados, el uso de un diccionario muy extenso podría no ser factible. Además, vocabularios muy extensos podrían incurrir en problemas de sobreajuste, generando descriptores muy especificos y perdiendo así la esencia del modelo BoW. </font> 
 
 > <font color = 'blue'>Llegados a este punto me gustaría comentar un aspecto importante. El diccionario con 200 palabras tarda un tiempo medio de indexación o busqueda (query_time) de alrededor de 11 segundos. Esto multiplicado por 500 imágenes de query son 5500 segundos, es decir, 90 minutos aproximadamente. Esto ha provocado que la ejecución de este modelo sea muy larga, tardando en torno a 2 horas. Esto es debido a que el tamaño del índice no es lo suficientemente grande como para generar unas listas de imágenes más dispersas. Así pues, se crea una gran acumulación de imágenes por palabra visual y se tarda más en procesar dicha información.</font> 
 
 > - Analyze the effect of the **vocabulary_size** in terms of mAP and average response time (train and query times). Are these times constant for each vocabulary? Some plots here can be useful to justify your answer.
 
 > <font color = 'blue'>Como ya se ha comentado, a cuantas más palabras tenga el vocabulario, mejor mAP se obtiene. También se ha comentado que a cuantas más palabras, menor es el tiempo de query o indexación requerido. No obstante, mayor es el tiempo de entrenamiento también.</font> 
 
 > <font color = 'blue'>En cuanto a la estabilidad en los tiempos de indexación, a mayor número de palabras más estable son los tiempos. Esto se ve reflejado en la desviación estándar de dichos tiempos. En el primer caso, con 200 palabras, la desviación estándar se situa en torno a un 35% respecto de la media. Este valor decrece monótonamente haste el último caso, con 200k palabras, donde esta se siua en torno al 27%.</font> 
 
 > - Do the results obtained depend on the set of images used to generate the vocabulary? How can we improve the retrieval performance?
 
 > <font color = 'blue'>Los resultados sí que se pueden ver afectados por el conjunto de imágenes empleado para generar las palabras visuales. Como ya se ha comentado, las palabras visuales se crear a partir de un proceso de clustering en el espacio de descriptores del conjunto de entrenamiento. Otro conjunto distinto de entrenamiento daría lugar a palabras distintas y por ende a un fichero inverso distinto. Ello concluiría con resultados distintos el la búsqueda de imágenes.</font> 
 
 > <font color = 'blue'>Para mejorar el rendimiento de la búsqueda, se podrían generar palabras usando imágenes muy parecidas en contexto al caso particular de uso de este sistema. Es decir, si se sabe que las imágenes que se van a buscar son imágenes de bosques y playas, generar un vocabulario usando este tipo de imágenes. Sin embargo, ello desembocaría posiblemente en un sobreajuste del modelo. Si se quiere un vocabulario más genérico, lo ideal es generarlo con una gran cantidad de imágenes diversas, reflejando así la variedad de información existente, y con un número de palabras suficiente como para plasmar dicha variedad de información sin caer en un sobreajuste del modelo.</font>

In [21]:
################################################################################
# TODO:                                                                        #
# Write here the code required to answer the questions stated above. You can   #
# Add more cells at this point if you need it.                                 #
################################################################################

vocab_size = [200, 2000, 20000, 200000]
m_ap_bow = []
train_time_bow = []
query_times_bow = []
for size in vocab_size: 
    filename = 'clust/clust_flickr60_k{}.fvecs'.format(size)
    bow = BoW(filename)
    training_time_bow = bow.build_index(train_names, train_desc)
    total_res, m_ap, query_time = compute_mAP(query_names, query_desc, bow, gt_file = 'holidays/holidays_images.dat')
    m_ap_bow.append(m_ap)
    train_time_bow.append(training_time_bow)
    query_times_bow.append(query_time)
    print('Vocabulary size of {}: \n'.format(size))
    print('MAP: {} \n'.format(m_ap))
    print('Training time: {} secs.'.format(training_time_bow))
    print('Query time: {} +- {} secs.'.format(np.mean(query_time), np.std(query_time)))
    print('\n\n')
          

print('Printing variables: \n')
print('m_ap_bow: {} \n'.format(m_ap_bow))
print('train_time_bow: {} \n'.format(train_time_bow))
print('query_times_bow: {} \n'.format(query_times_bow))
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################


Vocabulary size of 200: 

MAP: 0.04831850557132996 

Training time: 11.336090087890625 secs.
Query time: 11.279020719051362 +- 3.2623391381912263 secs.



Vocabulary size of 2000: 

MAP: 0.4230062621759219 

Training time: 11.315753936767578 secs.
Query time: 4.207218329429627 +- 1.2884616408542982 secs.



Vocabulary size of 20000: 

MAP: 0.4595130696784457 

Training time: 16.377777099609375 secs.
Query time: 0.8069303660392761 +- 0.2611327911529182 secs.



Vocabulary size of 200000: 

MAP: 0.522391190501858 

Training time: 21.18097186088562 secs.
Query time: 0.11888259220123291 +- 0.040282420877753425 secs.



Printing variables: 

m_ap_bow: [0.04831850557132996, 0.4230062621759219, 0.4595130696784457, 0.522391190501858] 

train_time_bow: [11.336090087890625, 11.315753936767578, 16.377777099609375, 21.18097186088562] 

query_times_bow: [array([1.33781319e+01, 1.32177501e+01, 1.32065809e+01, 1.28959942e+01,
       1.27883551e+01, 1.52837741e+01, 1.30957990e+01, 1.30187941e+01,
    

 ### TF - IDF
 As a final task of this assignment, let's implement the TF-IDF scoring scheme. Modify the `BoW` class you wrote before to include TF-IDF weighting procedure:

In [70]:
class BoW_TfIdf(object):
    '''
    Class to implement the BoW model + Inverted File + TF-IDF Scoring scheme.
    '''

    def __init__(self, vocab_file):
        '''
        Class constructor. You should load the vocabulary and initialize other stuff 
        required to save the inverted file and the IDF terms.
        '''
        self.vocab = iu.load_visual_vocab(vocab_file)
        self.nwords = self.vocab.getTrainDescriptors()[0].shape[0]
        ############################################################################
        # TODO:                                                                    #
        # Complete this function to save the stuff you need for the inverted file  #
        ############################################################################
             
        init_dict = [(str(i), []) for i in range(self.nwords)] 
        self.inverted = dict(init_dict)

        ############################################################################
        #                                 END OF YOUR CODE                         #
        ############################################################################

    def build_index(self, img_names, img_descs):
        '''
        Build an index from a set of images. Essentially, for each image, you should
            search its descriptors in the index and fill the inverted file structure
            in consequence. Additionally, IDF for each word should be computed here.

        - img_names: An ordered list with the names of query images.
        - img_descs: A list containing numpy arrays. Each numpy array i corresponds 
            to the descriptors found at image i.
        '''
        ############################################################################
        # TODO:                                                                    #
        # Write this function according to the description given above.            #
        ############################################################################
        start = time.time()
        for name, descs in zip(img_names, img_descs): 
            matches = self.vocab.match(descs)
            idxs = [match.trainIdx for match in matches]
            idxs_u = np.unique(np.array(idxs)) #para evitar imagenes repetidas en el inverted file
            for idx in list(idxs_u):
                self.inverted[str(idx)].append(name)
        
        num_imgs = len(img_names)
        for entry in list(self.inverted.keys()):
            den = len(self.inverted[entry])
            idf = np.log(num_imgs/(den + 1)) # +1 to avoid zero division
            self.inverted[entry].append(idf) #el final de cada entrada es el IDF de esa VW
            
        stop = time.time()
        training_time = stop - start
        return training_time
        ############################################################################
        #                                 END OF YOUR CODE                         #
        ############################################################################

    def search_image(self, descs):
        '''
        Search an image in the index. Use the TF-IDF here when scoring the images.
    
        - descs: A numpy array. It is the set descriptors extracted 
            from the query image.

        RETURNS: 
        - An ordered list of similar images, e.g.: ['100101.jpg', '100202.jpg', ...]
        '''    
        ############################################################################
        # TODO:                                                                    #
        # Write this function according to the description given above.            #
        ############################################################################
        
        start = time.time()
        matches = self.vocab.match(descs)
        idxs = [match.trainIdx for match in matches]      
        init_tf_dict = [(str(idx), 0) for idx in np.unique(np.array(idxs))]
        tf_dic = dict(init_tf_dict)
        
        n_kj = descs.shape[0]
        for entry in list(tf_dic.keys()):
            n_ij = idxs.count(int(entry))
            tf_dic[entry] = n_ij/n_kj # TF
        
        counter = {}
        for idx in idxs: 
            tf = tf_dic[str(idx)]
            idf_ = np.array(self.inverted[str(idx)][-1:], dtype = float)
            retrieved_imgs = self.inverted[str(idx)][:-1] #evito el IDF
            for ret_img in retrieved_imgs: 
                if ret_img not in list(counter.keys()): 
                    counter.setdefault(ret_img,tf*idf_) #creo la entrada con valor tf*idf
                else: 
                    counter[ret_img] += tf*idf_ # TF-IDF 
                    

        values = np.array(list(counter.values()))
        imgs_names = list(counter.keys())
        index_sort = np.argsort(values.flatten())[::-1] #descending order 
        best_imgs = [imgs_names[int(i)] for i in index_sort]
        set_imgs = set(best_imgs)
    
        stop = time.time()
        query_time = stop - start
        assert len(set_imgs) == len(best_imgs), "Duplicated Images"
        return best_imgs, query_time
        #return counter
         
        ############################################################################
        #                                 END OF YOUR CODE                         #
        ############################################################################


 Using the vocabularies of 200, 2K, 20K and 200K visual words and the new `BoW_TfIdf` class, answer the following questions:

 > **Error** 
 
 > <font color = 'green'> Primero de todo me gustaría comentar un error que se produjo en la primera práctica entregada. En esta, el mAP del diccionario con 200 palabras era superior a 1. El único motivo por el cual podía estar sucediendo eso era que se devolviesen imágenes repetidas. De esta forma, el numerador de la métrica mAP podría ser superior a su denominador, el cual marca el número total de imágenes relevantes para una determianda petición. Dicho error era causado por la sentencia ``index_sort = np.argsort(values.flatten())[::-1]`` de la función ``search_image``. En un principio, el array ``values`` no se aplanaba a una sola dimensión. Al no hacer eso, dicho array constituía un vector fila con dimensiones (N,1). La función de ``np.argsort`` toma como valor por defecto el -1 en su argumento ``axis``. Esto significa que se devolvían los índices que ordenaban cada columna por separado. Al ser un vector fila, se devolvía un vecto columna de 0s, ya que el valor que ordena cada columna en un vector fila es el mismo valor de la columna. Este hecho generaba que se escogiese posteriormente siempre la misma imagen a devolver por el sistema. De esta forma para ciertas imágenes de petición se contaban más resultados relevantes (numerador del mAP) de los que realmente había (denominador). </font> 
 
 > **Questions**:
 >
 > - What is the mAP obtained for each visual vocabulary?
 
 > <font color = 'blue'>Finalmente, este último caso no es más que una adaptación del BoW implementado anteriormente. En este modelo, el voto de cada palabra visual se pondera empleando el índice TF-IDF. Dicho índice mide, por un lado, la frecuencia con la que aparece una palabra en la imagen y, por otro lado, la presencia de dicha palabra en las imágenes de la base de datos. De esta forma, una palabra con gran presencia en una imagen y que aparece en pocas imágenes en la base de datos, tendrá un índice TF-IDF grande, indicando que es una palabra muy específica de la imagen de petición.</font> 
 
 > - Compare the performances obtained on each case. Is a larger vocabulary size always better? Why or why not?
 
 > <font color = 'blue'>Se observan los mismo fenómenos que con el BoW original. A mayor tamaño de vocabulario, mejor es el mAP, más cortos son los tiempos de búsqueda y mayores son los tiempos de entrenamiento. En cuanto a si siempre es mejor utilizar vocabularios grandes, me reitero que dependerá de la aplicación a llevar a cabo,  de los recursos de memoria de los cuales se disponga y al problema del sobreajuste ya mencionado. </font>
 
 > - Analyze the effect of the **vocabulary_size** in terms of mAP and average response time (train and query times). Are these times constant for each vocabulary? Some plots here can be useful to justify your answer.
 
 > <font color = 'blue'> El mAP y los tiempos de entrenamiento y búsqueda entre vocabularios aparecen ya comentados en el apartado anterior. En cuanto a la variabilidad de los tiempos de búsqueda, esta se sitúa en torno a un 30% respecto del tiempo medio (veáse la desviación estándar impresa por cada vocabulario). Esta variabilidad es muy semejante a la que ya presentaban los modelos de BoW originales. </font> 
 
 > - Do the results obtained depend on the set of images used to generate the vocabulary? How can we improve the retrieval performance?
 
 > <font color = 'blue'>Misma respuesta que en el anterior caso con el BoW original.</font> 
 
 > - How does TF-IDF affect the performance? Better or worse? Does this make sense?
 
 > <font color = 'blue'> En un principio, y bajo un punto de vista teórico,  TF-IDF debería permitir un rendimiento mayor en la búsqueda de imágenes. Como ya se ha comentado, esto se lograría otorgando un peso mayor a aquellas imágenes con particularidades semejantes a la imagen de petición, y dando menos importancia al voto de aquellas imágenes cuyo emparejamiento con al imagen de petición se debe a rasgos muy generales y/o poco discriminativos.</font> 
 
 > <font color = 'blue'>En esta práctica se puede contemplar que el mAP de los tres primeros vocabularios (200, 2K y 20K) es ligeramente mayor al obtenido usando el modelo BoW original. Curiosamente esto no es así con el vocabulario más extenso, el de 200K palabras, el cual presente un mAP ligeramente menor (una diferencia de aproximandamente 0,025). Es cierto que a cuantas más palabras, menor puede ser el TF (debido a que la frecuencia de la palabra en la imagen puede ser menor mientras que el número de palabras en esta se mantiene constante) pero mayor el IDF (mayor dispersión en el índice puede generar denominadores pequeños, mientras que el número de imágenes totales se mantiene constante en el numerador). Por lo tanto, no se me ocurre una razón evidente por la pueda suceder esto. Lo más seguro se deba a un problema de implementación.</font> 
 
 > <font color = 'blue'>En cuanto a tiempos de entrenamiento y búsqueda, lógicamente el TF-IDF tarda más. Esto es así ya que durante el entrenamiento se calcula el IDF de cada palabra y durante la búsqueda se determina el TF, procesos que en el BoW original no están presentes. Para los vocabularios on menos palabras estos procesos no tardan mucho, por lo que las diferencias en dichos tiempos no son muy elevadas. Sin embargo, para el diccionario de 200K palabras sí que se nota este hecho. Los tiempos de entrenamiento en este último se disparan a casi 6 veces los del BoW original, y los tiempos de búsqueda se duplican.</font> 
 
 > <font color = 'blue'>Teniendo en cuenta estos aspectos, y osbervando los resultados obtenidos en esta práctica, no se pueden concluir que el TF-IDF ofrezca mejoras considerables en los modelos BoW. Por un lado, las mejoras en los mAP no son muy llamativas e incluso puede no haber mejora, como el caso del diccionario de 200K palabras. Por otro lado, los tiempos de entrenamiento y búsqueda aumentan. Este efecto tiene aun más repercusión cuantas más palabras tenga el diccionario.</font>

##### Primero una prueba con el modelo de 200 palabras

In [71]:
filename = 'clust/clust_flickr60_k{}.fvecs'.format(200)
bow_tfidf = BoW_TfIdf(filename)
training_time_bow_t = bow_tfidf.build_index(train_names, train_desc)

In [72]:
total_res, m_ap_t, query_time_t = compute_mAP(query_names, query_desc, bow_tfidf, gt_file = 'holidays/holidays_images.dat')

In [76]:
print('Vocabulary size of {}: \n'.format(200))
print('MAP: {} \n'.format(m_ap_t))
print('Training time: {} secs.\n'.format(training_time_bow_t))
print('Query time: {} +- {} secs.'.format(np.mean(query_time_t), np.std(query_time_t)))
print('\n\n')

Vocabulary size of 200: 

MAP: 0.08537960523563595 

Training time: 12.27954888343811 secs.

Query time: 14.222891308784485 +- 4.111967012733803 secs.





##### El resto de vocabularios

In [77]:
################################################################################
# TODO:                                                                        #
# Write here the code required to answer the questions stated above. You can   #
# Add more cells at this point if you need it.                                 #
################################################################################

m_ap_bow_tfidf = []
train_time_bow_tfidf = []
query_times_bow_tfidf = []
vocab_size_ = [2000, 20000, 200000]
for size in vocab_size_: 
    print('Vocabulary size of {}: \n'.format(size))
    filename = 'clust/clust_flickr60_k{}.fvecs'.format(size)
    bow_tfidf = BoW_TfIdf(filename)
    training_time_bow_t = bow_tfidf.build_index(train_names, train_desc)
    total_res, m_ap_t, query_time_t = compute_mAP(query_names, query_desc, bow_tfidf, gt_file = 'holidays/holidays_images.dat')
    res = compute_mAP(query_names, query_desc, bow_tfidf, gt_file = 'holidays/holidays_images.dat')
    m_ap_bow_tfidf.append(m_ap_t)
    train_time_bow_tfidf.append(training_time_bow_t)
    query_times_bow_tfidf.append(query_time_t)
    print('MAP: {} \n'.format(m_ap_t))
    print('Training time: {} secs.\n'.format(training_time_bow_t))
    print('Query time: {} +- {} secs.'.format(np.mean(query_time_t), np.std(query_time_t)))
    print('\n\n')
    
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

Vocabulary size of 2000: 

MAP: 0.43212370655112464 

Training time: 11.264049053192139 secs.

Query time: 5.41852991771698 +- 1.6649846774186738 secs.



Vocabulary size of 20000: 

MAP: 0.4740974379248038 

Training time: 18.268032789230347 secs.

Query time: 1.2320635604858399 +- 0.4482588214361656 secs.



Vocabulary size of 200000: 

MAP: 0.4976056966502058 

Training time: 122.20123171806335 secs.

Query time: 0.2757780303955078 +- 0.09117671754715873 secs.



